## Accompanying code to 'Uncertainty Risk Parity' https://ssrn.com/abstract=3406321
### Example of uncertain risk parity and uncertain risk contributions

In [1]:
import numpy as np
import lib_risk_parity as rp

# 0 - Get estimates - C, covC, (to check out of sample) Ctest

In [2]:
import pandas as pd

fname = 'etf_weekly_rets.csv'
retsdf = pd.read_csv(fname, header=0, index_col=0) # (n x T) cols are old -> new
ids = retsdf.index
rets = np.matrix(retsdf)
#del retsdf

In [3]:
def test_train_mask(T, maskend=True, f=.75):
    # creates mask to select train/test entries
    # T = # periods in mask
    # maskend = if True, put all False values at end
    #           otherwise, at every k'th entry
    # f = fraction of values to be False
    # returns (length T) np.array with fraction f of values False, the rest True
    mask = np.full(T, True)
    if maskend:
        mask[round(f*T):] = False
    else:
        step = round(1/(1-f))
        for i in range(step-1,T,step):
            mask[i] = False
    return mask

def scale_to_median_volatility(r):
    # Function finds scalars so all series have the same volatility, the median of the unscaled volatilities
    # This is to create levered fixed income series that match equity vol
    # r = (n x T) returns series
    # returns: (length n) inflation needed for each series
    sd = np.diag(np.cov(r))**0.5
    return np.median(sd) / sd

def est_cov_of_cov_disjoint_periods(r, numsets=10):
    # function, in ad-hoc fashion, estimates covariance of covariance
    # as covariance of covariance estimates from disjoint periods
    # r = (n x T) returns
    # numsets = (int) number of sets to break r into
    # returns (n x n x n x n) covariance of covariance entries
    n, T = r.shape
    l = int(np.floor(T/numsets))  # num periods in each set
    M = np.zeros((n**2, numsets)) # holds each sets cov as a vector
    for i in range(numsets):
        M[:,i] = np.cov(r[:,i*l:(i+1)*l]).reshape([-1])  # store covariance over interval as a vector
    return np.cov(M).reshape((n,n,n,n))  # take cov of cov estimates, reshape from (n^2 x n^2) to (n x n x n x n)

def cov2corr(C):
    # converts covariance matrix to correlation matrix
    # C = (N x N) covariance matrix
    # returns (N x N) correlation matrix
    _invsd = 1./np.diag(C)**0.5
    return (C * _invsd).T * _invsd

def print_sd_cor(C, ids, d=2):
    n= C.shape[0]
    M = np.zeros((n,n+1))
    M[:,0] = np.diag(C) ** 0.5
    M[:,1:] = cov2corr(C)
    cols = ['sd']
    cols.extend(ids)
    df = pd.DataFrame(np.around(M,d), index=ids, columns=cols)
    return df

In [4]:
# split into train and test periods
maskend=True   # to make test set the series' end
#maskend=False  # to mask test set every kth period
mask = test_train_mask(T=rets.shape[1], maskend=maskend, f=.75)
rets_train = rets[:,mask]
rets_test = rets[:,~mask]

print("earliest-latest period end")
_w = retsdf.columns[mask]
_w1 = retsdf.columns[~mask]
print('train', _w[0],'-',_w[-1])
print('test', _w1[0],'-',_w1[-1])

# rescale all series to have same volatility on training set. This is to lever bonds to equity vol
leverReturns=False
if leverReturns:
    m = scale_to_median_volatility(rets_train)
    rets_train = np.multiply(rets_train.T,m).T
    rets_test = np.multiply(rets_test.T,m).T
    print('series multipliers to reach median volatility on train\n',np.around(m,2))

earliest-latest period end
train 8/30/2017 - 12/11/2019
test 12/18/2019 - 9/16/2020


In [5]:
# Ctest = (n x n) estimated cov over test portion
# C = (n x n) estimated cov over training portion
# covC = (n x n x n x n) estimated cov of cov over training portion
Ctest = np.cov(rets_test)
C = np.cov(rets_train)
covC = est_cov_of_cov_disjoint_periods(rets_train, numsets=12)

In [6]:
_df = print_sd_cor(C, ids)
#print(_df.to_latex())
print('training set')
_df

training set


,sd,AGG,EEM,EFA,SPY
AGG,0.41,1.00,-0.19,-0.19,-0.19
EEM,2.52,-0.19,1.00,0.89,0.78
EFA,1.73,-0.19,0.89,1.00,0.87
SPY,1.83,-0.19,0.78,0.87,1.00


In [7]:
_df = print_sd_cor(Ctest, ids)
#print(_df.to_latex())
print('test set')
_df

test set


,sd,AGG,EEM,EFA,SPY
AGG,1.75,1.00,0.69,0.74,0.53
EEM,4.82,0.69,1.00,0.93,0.84
EFA,4.52,0.74,0.93,1.00,0.89
SPY,4.18,0.53,0.84,0.89,1.00


# 1 - Construct portfolios

In [8]:
n = C.shape[0]
target = np.full(n, 1./n)  # target is equal risk contributions

In [9]:
w0 = rp.std_risk_parity(target=target, C=C)
w = rp.uncertain_risk_parity(target=target, C=C, covC=covC)

# 2 - Results

In [10]:
def print_results(ids, w, Ctest, C, covC=None, d=2):    
    M = np.zeros((4,n))
    M[0,:] = w
    M[1,:] = w * C.dot(w)     # variance contributions on train set
    M[3,:] = w * Ctest.dot(w) # variance contributions on test set
    if covC is not None:
        covv = np.einsum('i,k,ijkl',w,w,covC)  # cov of variance contributions on train set
        M[2,:] = np.diag(covv) ** 0.5  # standard deviation of variance contributiosn
    df = pd.DataFrame(np.around(M,d), columns=ids)
    df.index=('weight', 'v train', '+/-','v test')
    return df

In [11]:
print('std risk parity')
_df = print_results(ids, w0, Ctest, C)
#print(_df.to_latex())
_df

std risk parity


,AGG,EEM,EFA,SPY
weight,0.75,0.07,0.09,0.09
v train,0.06,0.06,0.06,0.06
+/-,0.00,0.00,0.00,0.00
v test,2.67,0.62,0.86,0.66


In [12]:
print('uncertain risk parity')
_df = print_results(ids, w, Ctest, C, covC=covC)
#print(_df.to_latex())
_df

uncertain risk parity


,AGG,EEM,EFA,SPY
weight,0.79,0.06,0.08,0.07
v train,0.08,0.05,0.05,0.04
+/-,0.08,0.67,0.54,0.61
v test,2.77,0.52,0.72,0.45
